# Ausreißer (Boxplot)
Wir erstellen mit ein paar Beispielwerten ein Boxplot-Diagramm mit Matplotlib.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

daten = [3, 9, 17, 23, 27, 31, 42, 48, 57, 65]
__=plt.boxplot(daten)

Wir geben die Quantile aus:

In [ ]:
import numpy as np
np.quantile(daten, [0.25, 0.5, 0.75])

Wir wenden dies auf den Datensatz *cars* an und erstellen Boxplot-Diagramme für die Verbräuche (*mpg*) für die Kategorie *cyl* (Anzahl Zylinder).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as m

url = "https://raw.githubusercontent.com/troescherw/datasets/master/cars.csv"
cars = pd.read_csv(url, delimiter=";")

cyl4 = cars.loc[cars.cyl==4]
cyl6 = cars.loc[cars.cyl==6]
cyl8 = cars.loc[cars.cyl==8]

 
box_plot_data=[cyl4.mpg,cyl6.mpg,cyl8.mpg]
plt.boxplot(box_plot_data, labels=["4 Zylinder", "6 Zyliner", "8 Zylinder"])
plt.title("Verbrauch in Milen pro Gallonen nach Anzahl Zylinder")
plt.ylabel("Miles per Gallon")
plt.show()

Tipp: Mit *seaborn* können auf einfachere Weise ansehnliche Boxplot-Diagramme erstellt werden.

In [ ]:
import seaborn as sns
_=sns.boxplot(x=cars.cyl, y=cars.mpg, data=cars).set_title("Verbrauch in Milen pro Gallonen nach Anzahl Zylinder")

Wir entfernen nun Ausreißer, in dem wir alle Daten, die mehr als 1,5 * dem IQR vom unteren bzw. oberen Quartil entfernt sind, aus dem Datensatz entfernen. Verwenden wir hierzu die Autos mit 8 Zylindern:

In [ ]:
autos8cyl = cars.loc[cars.cyl==8]
sns.boxplot(y=autos8cyl.mpg)

In [ ]:
# Ermittle unteres und oberes Quartil

print("Alle mpg-Daten:")
print(autos8cyl.mpg)
q1 =  autos8cyl.mpg.quantile(q=.25)
q3 =  autos8cyl.mpg.quantile(q=.75)

iqr = q3 - q1
print("IQR=", iqr)

# Ausgabe der Ausreisser
print("Untere Ausreisser: \n", autos8cyl.mpg[(autos8cyl.mpg < q1 - 1.5*iqr)])
print("")
print("Obere Ausreisser: \n", autos8cyl.mpg[(autos8cyl.mpg > q3 + 1.5*iqr)])


df_no_outliers = autos8cyl.mpg[(autos8cyl.mpg > q1 - 1.5*iqr) & (autos8cyl.mpg < q3 + 1.5*iqr)]

print("Bereinigte Daten ohne Ausreißer:")
df_no_outliers


## Ausreißer ermitteln mit Hilfe der $\sigma$-Methode
Alle Werte, die (zum Beispiel) mehr als 2 Standardabweichungen "entfernt" liegen, werden als Ausreißer definiert (man kann natürlich auch andere Faktoren für die Standardabweichung einsetzen). Man spricht dann von der $2\sigma$-Methode bzw. von der $3\sigma$-Methode.

Mit der Funktion *zscore* ermitteln wir die Z-Werte der Daten. Wir benötigen den Absolutwert, da uns nur die Entfernung interessiert - egal ob die Daten zu groß oder zu klein sind. Durch Vergleich mit z.B. 2 ($2\sigma$) ergeben sich True- bzw.Falsewerte. Damit selektieren wir die Daten im DataFrame.

In [ ]:
from scipy import stats
import pandas as pd

df = pd.DataFrame({"Daten":[5,4,2,4,5,7,1,4,150,4,3,2,4]}) # 150 = Ausreißer
df[np.abs(stats.zscore(df)) < 2]


## LOF
Eine häufig angewandte Größe, um Ausreißer zu bestimmen, nennt sich LOF (Local Outlier Factor). Hier wird die "Dichte" der Daten ermittelt, also die Anzahl der in einer bestimmten Umgebung anzutreffenden Datenpunkte. Ist ein Punkt zu weit von diesen "Clutern" entfernt, also die Dichte zu gering, so liegt ein Ausreißer vor.

Hier verwenden wir hierfür die Klasse *LocalOutlierFactor*. Wir erstellen ein Objekt und übergeben die Methode *fit_predict* unsere Daten. Im Modell wird eine 1 gespeichert, falls es sich um einen Ausreißer handelt, ansonsten eine -1.

Standardmäßig werden Cluster von mindestens 20 Objekten erwartet. Da unser Beispiel viel weniger Datenpunkte hat, setzen wir hierfür den Wert auf 5.

In [ ]:
import seaborn as sns
df = pd.DataFrame(np.array([[0,1], [1,1], [1,2], [1,3], [0,2],[2,2], [6,7]]), columns = ["x", "y"])
print(df)
_=sns.scatterplot(x=df.x, y=df.y)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

outlier_model = LocalOutlierFactor(n_neighbors=5).fit_predict(df)
print("Ergebnis: 1=Kein Ausreißer, -1 = Ausreißer: ", outlier_model)
print("Keine Ausreißer:")
print(df[outlier_model==1])
print("Ausreißer:")
print(df[outlier_model==-1])

# Scatterplot mit farblicher Darstellung des Ausreissers
_=sns.scatterplot(x=df[outlier_model==1].x, y=df[outlier_model==1].y)
_=sns.scatterplot(x=df[outlier_model==-1].y, y=df[outlier_model==-1].y, color="r", label="Ausreißer")
